In [8]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import functools as ft
import random

## Tests

In [9]:
#Missing Values:
def TestMissingValues(dates):
    k = 0
    fails = 0
    date_now = dates[0]
    date_next = dates[1]
    for i in range(1, len(dates)-1):
        if date_now + timedelta(hours = 1) != date_next:
            fails += 1
            if k == 0:
                print("TestMissingValues Failed! \nGaps: \n")
            print(date_now, "and", date_next)
            k = 1
            if fails > 50:
                print("(...)")
                print("Wowe such bad")
                break
        date_now = dates[i]
        date_next = dates[i+1]
    if k == 0:
        print("No missing values!")
        return True
    
    return False

#Duplicates
def TestNoDuplicates(df, group_by = "Date"):
    groups = df.groupby(by = group_by)[group_by].count()
    if len(groups[groups!=1]) != 0:
        print("TestNoDuplicates Failed!")
    else:
        print("TestNoDuplicates Success!")

#Nan
def TestNoNANs(df, col):
    if df[col].isnull().values.any():
        print("TestNoNANs Failed!")
    else:
        print("TestNoNANs Success!")

#lagged variables
def TestLags(df, size_of_test=1000):
    def perform_test(df, prev_date, lag_val):
        prev_val = df[df['DateUTC'] == prev_date]['PriceMWh'].values
        if prev_val.size > 0:
            if prev_val[0] != lag_val[0]:
                return False

    if size_of_test == 'All': # Warning: takes very long!
        dates = df['DateUTC']
    else:
        index = random.sample(range(0, len(df)), size_of_test)
        dates = df['DateUTC'].iloc[index]
        
    tests = [True, True, True]
    
    for date in dates:
        cur_row = df[df['DateUTC'] == date]
        lag1 = cur_row['Lag1'].values
        lag2 = cur_row['Lag2'].values
        lag7 = cur_row['Lag7'].values

        # test for Lag1
        prev_date1 = date - timedelta(1)
        if perform_test(df, prev_date1, lag1) == False:
            tests[0] = False

        # test for Lag2
        prev_date2 = date - timedelta(2)
        if perform_test(df, prev_date2, lag2) == False:
            tests[1] = False
                
        # test for Lag7
        prev_date7 = date - timedelta(7)
        if perform_test(df, prev_date7, lag7) == False:
            tests[2] = False
    
    if all(tests):
        print('TestLag Success!')
    else:
        print('TestLag Failed!')
        print(f'Lag1 {"Success" if tests[0] else "Failed"}')
        print(f'Lag2 {"Success" if tests[1] else "Failed"}')
        print(f'Lag7 {"Success" if tests[2] else "Failed"}')
        print(count)

#Previous day lag
def TestPrevDayLag(df, size_of_test=50):
    if size_of_test == 'All': # Warning: takes very long!
        dates = df['DateUTC']
    else:
        index = random.sample(range(0, len(df)), size_of_test)
        dates = df['DateUTC'].iloc[index]
    
    test = True
    
    for date in dates:
        cur_row = df[df['DateUTC'] == date]
        prev_date = (date - timedelta(1)).date()
        for hour in range(0,24):
            prev_val = df[(df['Date']==prev_date) & (df['Hour']==hour)]['PriceMWh'].values
            hour_val = cur_row[f'{hour}'].values
            if prev_val[0] != hour_val[0]:
                test = False
            
    if test:
        print('TestPrevDayLag Success!')
    else:
        print('TestPrevDayLag Failed!') 

In [10]:
NP_price = pd.read_csv("data/NP_2013-18.csv")
NP_price.rename(columns = {' Price':'Price', ' Grid load forecast':'Grid load forecast',
                            ' Wind power forecast': 'Wind power forecast'}, inplace = True)
NP_price['DateTime'] = pd.to_datetime(NP_price['Date'], format="%Y-%m-%d %H:%M:%S", utc=True)
NP_price['Date'] = NP_price['DateTime'].apply(lambda x: x.date())
NP_price['Time'] = NP_price['DateTime'].apply(lambda x: x.time())
NP_price['Hour'] = NP_price['Time'].apply(lambda x: x.hour)
NP_price['Lag_date_0'] = NP_price['Date'].copy()
NP_price['Lag_date_1'] = NP_price['Date'].apply(lambda x: x-timedelta(1))
NP_price['Lag_date_2'] = NP_price['Date'].apply(lambda x: x-timedelta(2))
NP_price['Lag_date_3'] = NP_price['Date'].apply(lambda x: x-timedelta(3))
NP_price['Lag_date_7'] = NP_price['Date'].apply(lambda x: x-timedelta(7))    
#NP_price['Price_Lag1'] = NP_price['Price'].shift(24)
#NP_price['Price_Lag2'] = NP_price['Price'].shift(24*2)
#NP_price['Price_Lag3'] = NP_price['Price'].shift(24*2)
#NP_price['Price_Lag7'] = NP_price['Price'].shift(24*7)
#NP_price['Grid load forecast_2'] = NP_price['Grid load forecast']**2
#NP_price['Wind power forecast_2'] = NP_price['Wind power forecast']**2
#NP_price['Grid load forecast_Lag1'] = NP_price['Grid load forecast'].shift(24)
#NP_price['Grid load forecast_Lag7'] = NP_price['Grid load forecast'].shift(24*7)
#NP_price['Wind power forecast_Lag1'] = NP_price['Wind power forecast'].shift(24)
#NP_price['Wind power forecast_Lag7'] = NP_price['Wind power forecast'].shift(24*7)

for val in ['Price', 'Grid load forecast', 'Wind power forecast']:
        if val in [ 'Grid load forecast', 'Wind power forecast']:
            wide_df = pd.pivot(NP_price, index='Date', columns='Time', values = val)
            wide_df.columns = [val + "_h" + f'{i}' for i in range(0,24)]
            wide_df["Lag_date_0"] = wide_df.index
            NP_price = NP_price.merge(wide_df, on='Lag_date_0')
            wide_df.columns = [val + "_lag1_h" + f'{i}' for i in range(0,24)] +['Lag_date_1']
            NP_price = NP_price.merge(wide_df, on='Lag_date_1')
            wide_df.columns = [val + "_lag7_h" + f'{i}' for i in range(0,24)] + ["Lag_date_7"]
            NP_price = NP_price.merge(wide_df, on='Lag_date_7')
            NP_price = NP_price.reset_index(drop=True)
        else:
            wide_df = pd.pivot(NP_price, index='Date', columns='Time', values = val)
            wide_df.columns = [val + "_lag1_h" + f'{i}' for i in range(0,24)] 
            wide_df['Lag_date_1'] = wide_df.index
            NP_price = NP_price.merge(wide_df, on='Lag_date_1')
            
            wide_df.columns = [val + "_lag2_h" + f'{i}' for i in range(0,24)] + ["Lag_date_2"]
            NP_price = NP_price.merge(wide_df, on='Lag_date_2')

            wide_df.columns = [val + "_lag3_h" + f'{i}' for i in range(0,24)]+ ["Lag_date_3"]
            NP_price = NP_price.merge(wide_df, on='Lag_date_3')

            wide_df.columns = [val + "_lag7_h" + f'{i}' for i in range(0,24)] + ["Lag_date_7"]
            wide_df['Lag_date_7'] = wide_df.index
            NP_price = NP_price.merge(wide_df, on='Lag_date_7')
            
            NP_price = NP_price.reset_index(drop=True)

NP_price['Week day'] = NP_price['DateTime'].dt.weekday
NP_price = pd.get_dummies(NP_price, columns=["Week day"])
NP_price = NP_price.dropna().reset_index(drop = True)

print(f'Number of columns in table: {len(NP_price.columns.values)}')
TestMissingValues(NP_price["DateTime"])
TestNoDuplicates(NP_price, group_by = "DateTime")
TestNoNANs(NP_price, "Price")

Number of columns in table: 259
No missing values!
TestNoDuplicates Success!
TestNoNANs Success!


In [11]:
NP_price.to_csv('data/full_NP_2018-13.csv', sep=';', index=False)

In [12]:
NP_price

,Date,Price,Grid load forecast,Wind power forecast,DateTime,Time,Hour,Lag_date_0,Lag_date_1,Lag_date_2,...,Wind power forecast_lag7_h21,Wind power forecast_lag7_h22,Wind power forecast_lag7_h23,Week day_0,Week day_1,Week day_2,Week day_3,Week day_4,Week day_5,Week day_6
0,2013-01-22,37.09,54113.0,2630.0,2013-01-22 00:00:00+00:00,00:00:00,0,2013-01-22,2013-01-21,2013-01-20,...,141.0,127.0,116.0,0,1,0,0,0,0,0
1,2013-01-22,36.94,53112.0,2570.0,2013-01-22 01:00:00+00:00,01:00:00,1,2013-01-22,2013-01-21,2013-01-20,...,141.0,127.0,116.0,0,1,0,0,0,0,0
2,2013-01-22,36.98,52698.0,2503.0,2013-01-22 02:00:00+00:00,02:00:00,2,2013-01-22,2013-01-21,2013-01-20,...,141.0,127.0,116.0,0,1,0,0,0,0,0
3,2013-01-22,37.10,52750.0,2440.0,2013-01-22 03:00:00+00:00,03:00:00,3,2013-01-22,2013-01-21,2013-01-20,...,141.0,127.0,116.0,0,1,0,0,0,0,0
4,2013-01-22,37.47,53483.0,2385.0,2013-01-22 04:00:00+00:00,04:00:00,4,2013-01-22,2013-01-21,2013-01-20,...,141.0,127.0,116.0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51907,2018-12-24,50.72,52591.0,2544.0,2018-12-24 19:00:00+00:00,19:00:00,19,2018-12-24,2018-12-23,2018-12-22,...,788.0,811.0,776.0,1,0,0,0,0,0,0
51908,2018-12-24,49.86,51787.0,2785.0,2018-12-24 20:00:00+00:00,20:00:00,20,2018-12-24,2018-12-23,2018-12-22,...,788.0,811.0,776.0,1,0,0,0,0,0,0
51909,2018-12-24,49.09,51488.0,2919.0,2018-12-24 21:00:00+00:00,21:00:00,21,2018-12-24,2018-12-23,2018-12-22,...,788.0,811.0,776.0,1,0,0,0,0,0,0
51910,2018-12-24,49.02,50928.0,3119.0,2018-12-24 22:00:00+00:00,22:00:00,22,2018-12-24,2018-12-23,2018-12-22,...,788.0,811.0,776.0,1,0,0,0,0,0,0
